----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Content - Streaming Price Cache - Synchronous calls

This notebook demonstrates how to use the Streaming Price interface to cache streaming data from the Refinitiv Data Platform.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the LD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [ ]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [1]:
import lseg.data as ld
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*lseg-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x10e686340 {name='workspace'}>

## Define and Open a cache for Streaming data
Define a Pricing object from the Streaming data feed - note the **get_stream()** call

In [3]:
# Define our Streaming Price object
streaming = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY='],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
streaming.open()
# When you want the current price, get a snapshot
streaming.get_snapshot()

Instrument     ASK     BID
0       EUR=   1.075  1.0746
1       GBP=  1.2729  1.2724
2       JPY=  157.92  157.87

#### Call get_snapshot() again, prices may change (assuming instruments are being traded/quoted)

In [4]:
snap = streaming.get_snapshot()
display(snap)

Instrument     ASK     BID
0       EUR=   1.075  1.0746
1       GBP=  1.2728  1.2724
2       JPY=  157.92  157.87


### Alternative ways of accessing instruments + values

#### Direct Access to fields
We can directly access the cache without the need to call **get_snapshot()**

In [5]:
# Directly access the latest 'BID' price for the EURO
streaming['EUR=']['BID']

1.0746

In [6]:
## Access the individual StreamingPrice object for GBP
gbp = streaming['GBP=']
# and then use object to access individual fields
gbp['ASK']

1.2728

In [7]:
# Will be different from above if price has changed
gbp['ASK']

1.2728

#### Iterate on fields

In [8]:
print('JPY=')
for field_name, field_value in streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	ASK : 157.92
	BID : 157.87


#### Iterate on Streaming instruments and fields

In [9]:
for instrument in streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	ASK : 1.0748
	BID : 1.0746
GBP=
	ASK : 1.2727
	BID : 1.2726
JPY=
	ASK : 157.92
	BID : 157.87


### Close the Streaming Items when no longer required

In [10]:
streaming.close()

<OpenState.Closed: 'Closed'>

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [11]:
# Define our Streaming Price object
mixed = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
state = mixed.open()
mixed.get_snapshot()

Instrument     ASK     BID
0       EUR=  1.0749  1.0747
1       GBP=  1.2729  1.2724
2     BADRIC    <NA>    <NA>

You can check the Status of any instrument, so lets check the invalid one

In [16]:
mixed['BADRIC'].code

'Closed'

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

## Close the session

In [17]:
ld.close_session()